In [1]:
import pandas as pd

# Configure and load data

In [2]:
# Host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [3]:
# import datasets from local files

import glob

# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')
df_stripe_payouts = pd.read_csv(f'data/{host}/{year}/stripe_payouts.csv')

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load paypal data
dir = f'data/{host}/{year}/paypal'
df_paypal = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_paypal = df_paypal.dropna(axis=1, how='all')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')


/var/folders/qf/j616b9d53yn77zcpky42s5r40000gn/T/ipykernel_73083/1724319478.py:10: DtypeWarning: Columns (20,28,29,30,31,32,33,40,41,42,43,45,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stripe_payouts = pd.read_csv(f'data/{host}/{year}/stripe_payouts.csv')


# Stripe transactions without paymentMethod

In [9]:
# Create a df_platform_stripe dataframe that is a merge of df_platform and df_stripe on merchantId on the left and id on the right
df_platform_stripe = pd.merge(df_platform, df_stripe, left_on='merchantId', right_on='id', how='right')

There are transactions on the platform with a stripe.id in merchantId that do not have STRIPE as the paymentMethod.service.

In [13]:
df_platform_stripe['paymentMethod.service'].value_counts(dropna=False)

paymentMethod.service
STRIPE            124584
NaN                38506
OPENCOLLECTIVE       806
Name: count, dtype: int64

This means that we can't use the paymentMethod.service as a qualifier of that a contribution was made with Stripe. 
We need to merge directly on merchantId without filtering.

Case study: find a stripe transaction that is registered on the platform without the reference to the stripe id.

In [14]:
# find record in df_platform_stripe where id is 209b4298-267d-4fdf-8ee1-167ec49c4765
# this is a contribution that is tracked on the platform
# it corresponds to stripe id ch_3N0XlpBYycQg1OMf18wE89HX
# however, this stripe id is not registered on the platform
# see transaction at https://opencollective.com/dashboard/opensource/host-transactions?openTransactionId=209b4298-267d-4fdf-8ee1-167ec49c4765&group=38c62837-9fe5-4399-8aa4-1b88ef98e40a

df_platform[df_platform['id'] == '209b4298-267d-4fdf-8ee1-167ec49c4765']

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,...,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
580387,209b4298-267d-4fdf-8ee1-167ec49c4765,209b4298-267d-4fdf-8ee1-167ec49c4765,CONTRIBUTION,NaN,NaN,NaN,38c62837-9fe5-4399-8aa4-1b88ef98e40a,CREDIT,Monthly financial contribution to Helix (backer),2023-04-24T22:22:07.997Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# How does the platform ledger handle refunds?